# scipy stats

This notebook focuses on the use of the scipy.stats module

It is built based on a learn-by-example approach So it only covers a little part of the module's functionalities but provides a practical application.

Some knowledge of `numpy` and `matplotlib` is needed to fully understand the content.

## Introduction

The scipy.stats module provides mainly:
* probability distributions: continuous, discrete and multivariate
* statistical functions such as statistics and tests

For further details you can check [the official documentation](https://docs.scipy.org/doc/scipy/reference/stats.html)

## Imports

In [ ]:
%matplotlib inline
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd

## Probability distributions

The scipy.stats module provides a very complete set of probability distributions.

There are three types of distributions:
* Continuous
* Discrete
* Multivariate

Each of the univariate types is inherited from the same class, so they all have a common API.

### Continuos distributions

There are ~100 different continuous distributions. Some of the methods in the API:
* `cdf`: Cumulative Distribution Function
* `pdf`: Probability Density Function
* `rvs`: Random Variable Sample
* `ppf`: Percent Point Function (inverse of the CDF)
* `fit`: return MLE estimations of location, scale and shape, given a set of data

In [ ]:
N_SAMPLES = 1000

pds = [('Normal', stats.norm(), (-4., 4.)), 
      ('LogNormal', stats.lognorm(1.), (0., 4.)),
      ('Students T', stats.t(3.), (-10., 10.)),
      ('Chi Squared', stats.chi2(1.), (0., 10.))]

n_pds = len(pds)
fig, ax_list = plt.subplots(n_pds, 3)
fig.set_size_inches((5.*n_pds, 10.))
for ind, elem in enumerate(pds):
    
    pd_name, pd_func, pd_range = elem

    x_range = np.linspace(*pd_range, 101)
    
    # Probability Density Function
    ax_list[ind, 0].plot(x_range, pd_func.pdf(x_range))
    ax_list[ind, 0].set_ylabel(pd_name)
    
    # Cumulative Distribution Function
    ax_list[ind, 1].plot(x_range, pd_func.cdf(x_range))
    ax_list[ind, 1].fill_between(x_range, pd_func.cdf(x_range))
    ax_list[ind, 1].set_ylim([0., 1.])
    
    # Random Variable Sample
    ax_list[ind, 2].hist(pd_func.rvs(size=N_SAMPLES), bins=50)
    
    if ind == 0:
        _ = ax_list[ind, 0].set_title('Probability Density Function')
        _ = ax_list[ind, 1].set_title('Cumulative Distribution Function')
        _ = ax_list[ind, 2].set_title('Random Sample')

## Discrete Distributions

Discrete distributions have quite the same API. Having `pmf`= Probability Mass Function (instead of pdf)

In [ ]:
N_SAMPLES = 1000

pds = [('Binomial', stats.binom(20, 0.7), (0., 21.)),
      ('Poisson', stats.poisson(10.), (0., 21.))]

n_pds = len(pds)
fig, ax_list = plt.subplots(n_pds, 3)
fig.set_size_inches((8.*n_pds, 8.))
for ind, elem in enumerate(pds):
    
    pd_name, pd_func, pd_range = elem

    x_range = np.arange(*pd_range)
    
    # Probability Mass Function
    ax_list[ind, 0].bar(x_range, pd_func.pmf(x_range))
    ax_list[ind, 0].set_ylabel(pd_name)
    
    # Cumulative Distribution Function
    ax_list[ind, 1].plot(x_range, pd_func.cdf(x_range))
    ax_list[ind, 1].fill_between(x_range, pd_func.cdf(x_range))
    ax_list[ind, 1].set_ylim([0., 1.])
    
    # Random Variable Sample
    ax_list[ind, 2].hist(pd_func.rvs(size=N_SAMPLES), bins=x_range - 0.5)
    
    if ind == 0:
        _ = ax_list[ind, 0].set_title('Probability Mass Function')
        _ = ax_list[ind, 1].set_title('Cumulative Distribution Function')
        _ = ax_list[ind, 2].set_title('Random Sample')

## Example: creating a financial product

### Load and manipulate the data

In [ ]:
df_prices = pd.read_csv('../resources/stock.csv')

In [ ]:
df_prices.head(10)

In [ ]:
_ = df_prices[['Apple', 'Microsoft']].plot(title='2016 stock prices')

In [ ]:
# Compute the daily relative increments
df_incs = df_prices.drop('Date', axis=1)
df_incs = ((df_incs - df_incs.shift(1))/df_incs.shift(1)).loc[1:, :]
df_incs['Date'] = df_prices.Date
df_incs.head(10)

In [ ]:
_ = df_incs[['Apple', 'Microsoft']].plot(title='2016 stock prices variations')

In [ ]:
m = np.mean(df_incs)
print(m)

In [ ]:
s = np.std(df_incs, ddof=1)
print(s)

In [ ]:
c = df_incs.cov()
c

## Create a Normal distribution

Let's assume that the stock prices follow a Normal distribution

In [ ]:
# we can use the fit method to get the MLE of the mean and the std
stats.norm.fit(df_incs.Apple)

In [ ]:
# Create estimated distributions based on the sample
app_dist = stats.norm(m['Apple'], s['Apple'])
win_dist = stats.norm(m['Microsoft'], s['Microsoft'])
intl_dist = stats.norm(m['Intel'], s['Intel'])

In [ ]:
# We can test if this data fits a normal distribution (Kolmogorov-Smirnov test)
app_KS = stats.kstest(df_incs['Apple'], 'norm', [m['Apple'], s['Apple']])
win_KS = stats.kstest(df_incs['Microsoft'], 'norm', [m['Microsoft'], s['Microsoft']])
intl_KS = stats.kstest(df_incs['Intel'], 'norm', [m['Intel'], s['Intel']])
print('''Apple: {}
Microsoft: {}
Intel: {}'''.format(app_KS, win_KS, intl_KS))

![Ummmmmm](../resources/homer-doh.jpg)

In [ ]:
# Compare histogram with estimated distribution
x_range = np.arange(-0.05, +0.0501, 0.001)
x_axis = (x_range[1:] + x_range[:-1])/2.
n_incs = df_incs.shape[0]
y_app = (app_dist.cdf(x_range[1:]) - app_dist.cdf(x_range[:-1]))*n_incs
y_win = (win_dist.cdf(x_range[1:]) - win_dist.cdf(x_range[:-1]))*n_incs
y_intl = (intl_dist.cdf(x_range[1:]) - intl_dist.cdf(x_range[:-1]))*n_incs

fig = plt.figure(figsize=(16., 6.))
ax_app = fig.add_subplot(131)
_ = ax_app.hist(df_incs['Apple'], bins=x_range, color='powderblue')
_ = ax_app.set_xlabel('Apple')
_ = ax_app.plot(x_axis, y_app, color='blue', linewidth=3)
ax_win = fig.add_subplot(132)
_ = ax_win.hist(df_incs['Microsoft'], bins=x_range, color='navajowhite')
_ = ax_win.set_xlabel('Microsoft')
_ = ax_win.plot(x_axis, y_win, color='orange', linewidth=3)
ax_intl = fig.add_subplot(133)
_ = ax_intl.hist(df_incs['Intel'], bins=x_range, color='lightgreen')
_ = ax_intl.set_xlabel('Intel')
_ = ax_intl.plot(x_axis, y_win, color='green', linewidth=3)

## Exercise:

Imagine you are a product designer in a finantial company. You want to create a new investment product to be "sold" to your clients based on the future stock prices of some IT companies. The profit the client gets from his investement is calculated like this:
* At  the time of the investment we check the initial stock prices
* 12 months later (let's say 240 work days), the client gets 100% of the investement back. Additionally if all stock prices are higher than the initial ones, the client earns half the lowest increment (in %). 

**What is the expected profit of this investment?**

**What is the 5% highest risk that the finantial company is assuming?**

First we will try to create a finantial product based on the stock prices of Apple and Microsoft

### Create a multinormal distribution

In [ ]:
# Create a multivariate normal distribution object
m_norm = stats.multivariate_normal(m[['Apple', 'Microsoft']], df_incs[['Apple', 'Microsoft']].cov())

In [ ]:
# Show the contour plot of the pdf
x_range = np.arange(-0.05, +0.0501, 0.001)
x, y = np.meshgrid(x_range, x_range)

pos = np.dstack((x, y))
fig_m_norm = plt.figure(figsize=(6., 6.))
ax_m_norm = fig_m_norm.add_subplot(111)
ax_m_norm.contourf(x, y, m_norm.pdf(pos), 50)
_ = ax_m_norm.set_xlabel('Apple')
_ = ax_m_norm.set_ylabel('Microsoft')

### Compute the expected profit and top 5% risk

In [ ]:
# Create N (e.g 1000) random simulations of the daily relative increments with 240 samples
N_SIMS = 1000
daily_incs = m_norm.rvs(size=[240, N_SIMS])

In [ ]:
# Calculate yearly increments (from the composition of the daily increments)
year_incs = (daily_incs + 1.).prod(axis=0)

In [ ]:
# calculate the amount payed for each simulation
def amount_to_pay(a):
    if np.all( a >= 1.):
        return (a.min() - 1)/2
    else:
        return 0.
earnings = np.apply_along_axis(amount_to_pay, 1, year_incs)

In [ ]:
_ = plt.hist(earnings, bins=50)

In [ ]:
print('Expected profit of the investment: {:.2%}'.format(earnings.mean()))

In [ ]:
# To compute the 5% higher profit use the stats.scoreatpercentile function
print('%5 higher profit of the investment: {:.2%}'.format(stats.scoreatpercentile(earnings, 95)))
print('%1 higher profit of the investment: {:.2%}'.format(stats.scoreatpercentile(earnings, 99)))

Both the expected profit and the risk assessed are too high!!

**Try adding Intel to the product in order to lower them down**

In [ ]:
# %load -r 2:10 solutions/07_02_scipy_stats.py